<h2 align = "center">Sprawozdanie mofit</h2>
<h4 align="center">Łukasz Puchała, Karol Grzywa</h4>

In [1]:
import numpy as np
from numba import njit
import math
import matplotlib.pyplot as plt
from math import sin, cos, pi, e, exp
from scipy.linalg import eigh
import Utilis

m = 0.067 # m0
h = 1
at2meV = 27211.6
omega = 10/at2meV
ab = 0.05292
j = 0 + 1j
at2s = 2.42e-5*1e-12

L = 100/ab
N = 2
a = L/(2*N)

In [2]:
grid_N2_L100 = Utilis.ReadFile('Files/wezly_N_2_L_100.dat')


<h2 align = "center">generowanie macierzy lokalnych </h2>

In [3]:
grid_N2_L100 = Utilis.ReadFile('Files/wezly_N_2_L_100.dat')
wezly = np.array(grid_N2_L100)

In [4]:
nlg_N2_l100 = Utilis.ReadFile('Files/nlg_N_2_L_100.dat.txt')
nlg = np.array(nlg_N2_l100)
nlg

array([[ 1,  1,  1],
       [ 1,  2,  2],
       [ 1,  3,  5],
       [ 1,  4,  6],
       [ 1,  5, 26],
       [ 1,  6, 27],
       [ 1,  7, 28],
       [ 1,  8, 29],
       [ 1,  9, 30],
       [ 2,  1,  2],
       [ 2,  2,  3],
       [ 2,  3,  6],
       [ 2,  4,  7],
       [ 2,  5, 31],
       [ 2,  6, 32],
       [ 2,  7, 27],
       [ 2,  8, 33],
       [ 2,  9, 34],
       [ 3,  1,  3],
       [ 3,  2,  4],
       [ 3,  3,  7],
       [ 3,  4,  8],
       [ 3,  5, 35],
       [ 3,  6, 36],
       [ 3,  7, 32],
       [ 3,  8, 37],
       [ 3,  9, 38],
       [ 4,  1,  4],
       [ 4,  2, 17],
       [ 4,  3,  8],
       [ 4,  4, 18],
       [ 4,  5, 39],
       [ 4,  6, 40],
       [ 4,  7, 36],
       [ 4,  8, 41],
       [ 4,  9, 42],
       [ 5,  1,  5],
       [ 5,  2,  6],
       [ 5,  3,  9],
       [ 5,  4, 10],
       [ 5,  5, 29],
       [ 5,  6, 43],
       [ 5,  7, 44],
       [ 5,  8, 45],
       [ 5,  9, 46],
       [ 6,  1,  6],
       [ 6,  2,  7],
       [ 6,  

In [5]:
# funkcja, co bierze poz. wezla naroznego elementu k o loklanym nr i
print( Utilis.xy_nlg(2, 5, nlg, wezly) )
# wspóółrzędne rzeczywiste punktu 
print( Utilis.xryr(k= 1, ksi1= 1, ksi2= -1, nlg= nlg, wezly= wezly) )
j,i,s = Utilis.Sloc(a)
print(s)
print(j)
print(i)

xd? -12.5 -50.0
(-12.5, -50.0)
xd? -50.0 -50.0
xd? -25.0 -50.0
xd? -50.0 -25.0
xd? -25.0 -25.0
(-25.0, -50.0)
[3967.508076735541, -991.8770191838856, -991.8770191838856, 247.96925479597147, 1983.7540383677706, -495.93850959194293, 1983.7540383677706, -495.93850959194293, 991.8770191838855, -991.8770191838856, 3967.5080767355403, 247.96925479597147, -991.8770191838856, 1983.754038367771, 1983.754038367771, -495.93850959194293, -495.93850959194293, 991.8770191838856, -991.8770191838856, 247.96925479597147, 3967.508076735541, -991.8770191838856, -495.93850959194293, -495.93850959194293, 1983.754038367771, 1983.7540383677706, 991.8770191838859, 247.96925479597147, -991.8770191838856, -991.8770191838856, 3967.508076735541, -495.93850959194293, 1983.754038367771, -495.93850959194293, 1983.7540383677713, 991.8770191838856, 1983.7540383677706, 1983.754038367771, -495.93850959194293, -495.93850959194293, 15870.03230694217, 991.8770191838859, 991.8770191838855, -3967.5080767355435, 7935.01615347

<h2>Wyznaczanie macierzy lokalnych</h2>

In [6]:
Sgiven = Utilis.ReadFile("./Files/S_loc_N2_L100.dat.txt")
Tgiven = Utilis.ReadFile("./Files/T_loc_N2_L100.dat.txt")
V11given = Utilis.ReadFile("./Files/V_loc_N2_L100_element_11.dat.txt")
jt,it,T = Utilis.Tmatrix(m,0.0001)
js,Is,S = Utilis.Sloc(a)
jv,iv,V = Utilis.Vkmatrix(11,a,m,omega,nlg,wezly)
print(V)
Tdif = T - np.array(Tgiven[2])
Sdiff = S - np.array(Sgiven[2])
Vdiff = V - np.array(V11given[2])


xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0
xd? 25.0 0.0
xd? 0.0 25.0
xd? 25.0 25.0
xd? 0.0 0.0


<h2>Rysunek odjętych od siebie macierzy wyznaczonych i danych</h2>

In [7]:
import plotly.express as px
px.imshow(Tdif.reshape((9, 9)), x=np.unique(it), y=np.unique(jt), title='Tdif', color_continuous_scale='Viridis').show()
px.imshow(Sdiff.reshape((9, 9)), x=np.unique(Is), y=np.unique(js), title='Sdiff', color_continuous_scale='Plasma').show()
px.imshow(Vdiff.reshape((9, 9)), x=np.unique(iv), y=np.unique(jv), title='Vdiff', color_continuous_scale='Blues').show()